In [2]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv("Co2_Es_5A.csv", low_memory=False)

In [3]:
# Rename columns to more meaningful names
rename_dict = {
    'Mk': 'Brand', 'Mp': 'Constructor', 'Mt': 'Test_mass', 'Ft': 'Energy', 'Fm': 'Fuel_mode',
    'm (kg)': 'Kg_veh', 'ec (cm3)': 'Engine_cm3', 'ep (KW)': 'Power_KW', 'z (Wh/km)': 'El_Consumpt_whkm',
    'Ewltp (g/km)': 'CO2_wltp', 'W (mm)': 'Wheelbase_mm', 'At1 (mm)': 'Axle_width_steer_mm', 
    'At2 (mm)': 'Axle_width_other_mm', 'T': 'Veh_type', 'Ve': 'Version', 'Cn': 'Veh_Model', 'Cr': 'Veh_Category','Fuel consumption ': 'Fuel consumption' 
}
df.rename(columns=rename_dict, inplace=True)

#Creating Eco-innovation column
# Create the 'Eco-innovation program' column based on 'IT' column's NaN status
df['Eco-innovation program'] = df['IT'].notna().astype(int)

# Dropping unnecessary columns
columns_to_drop = ['Man', 'Tan', 'r', 'Date of registration', 'Status', 'VFN', 'Va', 'Ct', 'IT', 'De', 'Vf','Mh', 'MMS']
df.drop(columns=columns_to_drop, inplace=True)

# Consolidating fuel types into simpler categories and converting text to lowercase
fuel_type_mapping = {
    'diesel/electric': 'hybrid diesel', 'petrol/electric': 'hybrid petrol', 'electric': 'electric',
    'diesel': 'diesel', 'petrol': 'petrol', 'lpg': 'lpg', 'ng': 'natural gas', 'hydrogen': 'hydrogen'
}

### filling NaNs
# Map and standardize the 'Energy' values, fill unmatched as 'Other' temporarily
df['Energy'] = df['Energy'].str.lower().map(fuel_type_mapping)

# Attempt to fill NaN values based on the group mode of 'Veh_Model' and 'Version'
df['Energy'] = df.groupby(['Veh_Model', 'Version'])['Energy'].transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan))


# Drop any rows that still have NaN values in the 'Energy' column
df.dropna(subset=['Energy'], inplace=True)

# print the DataFrame to confirm changes
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5377092 entries, 0 to 5409783
Data columns (total 26 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   ID                      int64  
 1   Country                 object 
 2   Constructor             object 
 3   Veh_type                object 
 4   Version                 object 
 5   Brand                   object 
 6   Veh_Model               object 
 7   Veh_Category            object 
 8   Kg_veh                  float64
 9   Test_mass               float64
 10  Enedc (g/km)            float64
 11  CO2_wltp                float64
 12  Wheelbase_mm            float64
 13  Axle_width_steer_mm     float64
 14  Axle_width_other_mm     float64
 15  Energy                  object 
 16  Fuel_mode               object 
 17  Engine_cm3              float64
 18  Power_KW                float64
 19  El_Consumpt_whkm        float64
 20  Ernedc (g/km)           float64
 21  Erwltp (g/km)           float64
 22 

In [4]:
## aligning formating for Brnad and Constructor
# More cleanup and type conversions
df['Brand'] = df['Brand'].str.upper().str.strip()
df['Constructor'] = df['Constructor'].str.upper().str.strip()

In [5]:
#Treating Constructor NaNs Mapping from the Brand
# Count the occurrences of NaN 'Constructor' values by 'Brand'
nan_constructors = df[df['Constructor'].isna()]
nan_counts_by_brand = nan_constructors['Brand'].value_counts().nlargest(50)

# Get the list of brands with the top 30 highest NaN counts
top_nan_brands = nan_counts_by_brand.index.tolist()

print(top_nan_brands)

brands_list = ['NISSAN', 'VOLVO', 'TOYOTA', 'MG', 'RENAULT', 'LAND ROVER', 'LYNK&CO', 'DACIA', 'MASERATI', 'FERRARI', 'JAGUAR', 'SUBARU', 'PEUGEOT', 'LEXUS', 'TESLA', 'INFINITI', 'SSANGYONG', 'VOLKSWAGEN', 'CITROEN', 'LOTUS', 'DFSK', 'CATERHAM', 'FIAT', 'LAMBORGHINI', 'ASTON MARTIN', 'AIWAYS', 'SECMA', 'BENTLEY', 'MORGAN', 'KIA', 'MERCEDES BENZ', 'CHEVROLET', 'AUDI', 'SUZUKI', 'MPM MOTORS', 'BLUECAR', 'HYUNDAI', 'FORD', 'MC LAREN', 'ALPINA', 'CADILLAC', 'BMW', 'SEAT', 'MOBILIZE', 'SKODA', 'MAZDA', 'PORSCHE', 'DS', 'HONDA', 'PGO']

# Initialize an empty dictionary to store the mappings
brand_to_constructor = {}

# Loop through each brand in the list
for brand in brands_list:
    # Filter df_Ptred for the current brand where 'Constructor' is not NaN
    filtered_df = df[(df['Brand'] == brand) & df['Constructor'].notna()]
    
    # Check if there are any non-NaN 'Constructor' values
    if not filtered_df.empty:
        # Find the most common 'Constructor' value for this brand
        most_common_constructor = filtered_df['Constructor'].mode()[0]
        brand_to_constructor[brand] = most_common_constructor
    else:
        # If no non-NaN 'Constructor' values are found, map the brand to None or some default
        brand_to_constructor[brand] = None

# Output the resulting dictionary
#print(brand_to_constructor)

# First, create a mapping series from the brand to constructor
brand_constructor_mapping = pd.Series(brand_to_constructor)

# Use 'map' to create a Series of constructors mapped from the 'Brand' column
mapped_constructors = df['Brand'].map(brand_constructor_mapping)

# Now fill NaN values in the 'Constructor' column using this mapped Series
df['Constructor'] = df['Constructor'].fillna(mapped_constructors)

# After attempting to fill with mapped constructors, fill remaining NaNs with the 'Brand' name
df['Constructor'] = df['Constructor'].fillna(df['Brand'])

# If you want to double-check or see the changes, you can display the DataFrame

#df.info()

['NISSAN', 'VOLVO', 'TOYOTA', 'SSANGYONG', 'SUBARU', 'MG', 'LAND ROVER', 'INFINITI', 'LYNK&CO', 'LEXUS', 'JAGUAR', 'PEUGEOT', 'VOLKSWAGEN', 'MASERATI', 'CITROEN', 'BERGADANA', 'DR', 'SEAT', 'BENIMAR', 'RENAULT', 'FIAT', 'FERRARI', 'GIOTTILINE', 'RODRIGUEZ LOPEZ AUTO', 'FORD', 'ADRIA', 'MERCEDES-BENZ', 'EUROGAZA', 'POLESTAR', 'CAPRON', 'PILOTE', 'CHALLENGER', 'MAHINDRA', 'OPEL', 'ROLLER TEAM', 'HYUNDAI', 'MC LOUIS', 'DFSK', 'BENTLEY', 'MCLOUIS', 'DREAMER', 'LAIKA CARAVANS SPA', 'CAPRON GMBH', 'RAPIDO', 'AIWAYS', 'DETHLEFFS', 'ASTON MARTIN', 'SUN LIVING', 'BMW', 'MERCEDES']


In [6]:
### NAN Processing for mass and width
# List of columns to fill NaN values with mean
columns_to_fill = ['Test_mass', 'Wheelbase_mm', 'Axle_width_steer_mm', 'Axle_width_other_mm']

# Grouping criteria
grouping_columns = ['Veh_Model']

# Applying the transformation
for column in columns_to_fill:
    # Compute the mean for each group and fill NaN values in the column
    df[column] = df.groupby(grouping_columns)[column].transform(lambda x: x.fillna(x.mean()))

# Check the results to ensure NaN values are filled
print(df[columns_to_fill].isna().sum())

# Remove rows that still contain NaN values in any of the four specified columns
df.dropna(subset=columns_to_fill, inplace=True)

# Verify the operation by checking for NaN values again
print("After cleaning:")
print(df[columns_to_fill].isna().sum())

Test_mass              126869
Wheelbase_mm              198
Axle_width_steer_mm       211
Axle_width_other_mm      2334
dtype: int64
After cleaning:
Test_mass              0
Wheelbase_mm           0
Axle_width_steer_mm    0
Axle_width_other_mm    0
dtype: int64


In [7]:
print(df.columns)  # This will show all column names

Index(['ID', 'Country', 'Constructor', 'Veh_type', 'Version', 'Brand',
       'Veh_Model', 'Veh_Category', 'Kg_veh', 'Test_mass', 'Enedc (g/km)',
       'CO2_wltp', 'Wheelbase_mm', 'Axle_width_steer_mm',
       'Axle_width_other_mm', 'Energy', 'Fuel_mode', 'Engine_cm3', 'Power_KW',
       'El_Consumpt_whkm', 'Ernedc (g/km)', 'Erwltp (g/km)', 'year',
       'Fuel consumption', 'Electric range (km)', 'Eco-innovation program'],
      dtype='object')


In [8]:
#NaN treatment 'Power_KW', 'El_Consumpt_whkm', 'Fuel consumption', 'Electric range (km)', 'Elect_Consumption', 'Erwltp (g/km) 
## averaging based on model

# Define grouping criteria
grouping_columns = ['Veh_Model', 'Version']
grouping_columns2 = ['Brand', 'Veh_type','Energy']
grouping_columns3 = ['Energy']
grouping_columns4 = ['Veh_type','Energy']
grouping_columns5 = ['Brand', 'Version']
grouping_columns6 = ['Veh_Model', 'Energy']
grouping_columns7 = ['Brand','Veh_type']


# Treat NaNs for 'Power_KW'
df['Power_KW'] = np.where(df['Energy'].isin(['diesel', 'petrol', 'lpg']),
                          df['Power_KW'].fillna(0),  # Fill NaN with 0 for 'diesel', 'petrol', 'lpg'
                          df.groupby(grouping_columns6)['Power_KW'].transform(lambda x: x.fillna(x.mean())))
                          # For other energy types, fill NaN with the mean of their group


# Treat NaNs for 'El_Consumpt_whkm'
df['El_Consumpt_whkm'] = np.where(df['Energy'].isin(['diesel', 'petrol', 'lpg']),
                                        df['El_Consumpt_whkm'].fillna(0),
                                        df.groupby(grouping_columns6)['El_Consumpt_whkm'].transform(lambda x: x.fillna(x.mean())))

# Treat NaNs for 'Fuel consumption'
df['Fuel consumption'] = np.where(df['Energy'] == 'electric',
                                        df['Fuel consumption'].fillna(0),
                                        df.groupby('Brand')['Fuel consumption'].transform(lambda x: x.fillna(x.mean())))

# Treat NaNs for 'Electric range (km)'
df['Electric range (km)'] = np.where(df['Energy'].isin(['diesel', 'petrol', 'lpg']),
                                          df['Electric range (km)'].fillna(0),
                                          df.groupby(grouping_columns4)['Electric range (km)'].transform(lambda x: x.fillna(x.mean())))


# Treat NaNs for 'CO2_wlpt'
df['CO2_wltp'] = np.where(df['Energy'] == 'electric',
                                        df['CO2_wltp'].fillna(0),
                                        df.groupby(grouping_columns6)['CO2_wltp'].transform(lambda x: x.fillna(x.mean())))

# Treat NaNs for 'Erwltp (g/km)'
df['Erwltp (g/km)'] = df['Erwltp (g/km)'].fillna(0)

# Treat NaNs for 'Engine_cm3'
df['Engine_cm3'] = df['Engine_cm3'].fillna(0)

# Check the results
print(df[['CO2_wltp','Power_KW', 'El_Consumpt_whkm', 'Fuel consumption', 'Electric range (km)', 'Erwltp (g/km)' , 'Engine_cm3']].isna().sum())

CO2_wltp               33266
Power_KW                7779
El_Consumpt_whkm       16388
Fuel consumption       20560
Electric range (km)    16708
Erwltp (g/km)              0
Engine_cm3                 0
dtype: int64


In [9]:
# Check if any NaNs remain and remove rows where any of these columns still have NaN
df.dropna(subset=['CO2_wltp', 'Power_KW', 'El_Consumpt_whkm', 'Fuel consumption', 'Electric range (km)', 'Erwltp (g/km)', 'Engine_cm3'], inplace=True)

# Verify that no NaN values are left in the specified columns



In [10]:
# Remove the specified columns
df.drop(columns=['Enedc (g/km)', 'Ernedc (g/km)','ID'], inplace=True)

# Remove rows with any NaN values
df.dropna(inplace=True)

# Optional: Check the result to ensure all NaN values are gone and columns are removed
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5168905 entries, 0 to 5409783
Data columns (total 23 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Country                 object 
 1   Constructor             object 
 2   Veh_type                object 
 3   Version                 object 
 4   Brand                   object 
 5   Veh_Model               object 
 6   Veh_Category            object 
 7   Kg_veh                  float64
 8   Test_mass               float64
 9   CO2_wltp                float64
 10  Wheelbase_mm            float64
 11  Axle_width_steer_mm     float64
 12  Axle_width_other_mm     float64
 13  Energy                  object 
 14  Fuel_mode               object 
 15  Engine_cm3              float64
 16  Power_KW                float64
 17  El_Consumpt_whkm        float64
 18  Erwltp (g/km)           float64
 19  year                    int64  
 20  Fuel consumption        float64
 21  Electric range (km)     float64
 22 

In [11]:
### removing brands and manufacturers wiht less than 0.1% share
def filter_by_frequency(df, column_name, threshold_pct=0.1):
    """ Filters the DataFrame by the frequency of occurrences in the specified column.
    
    Parameters:
        df (DataFrame): The DataFrame to filter.
        column_name (str): The column to calculate the frequency.
        threshold_pct (float): The percentage threshold to use for filtering (default is 0.1).
        
    Returns:
        DataFrame: A filtered DataFrame.
    """
    total_entries = len(df)
    threshold = (threshold_pct / 100) * total_entries
    value_counts = df[column_name].value_counts()
    valid_values = value_counts[value_counts >= threshold].index
    return df[df[column_name].isin(valid_values)]

# Apply the function to both 'Brand' and 'Constructor'
df = filter_by_frequency(df, 'Brand')
df = filter_by_frequency(df, 'Constructor')

# Optionally, check the result
print(df['Brand'].value_counts())
print(df['Constructor'].value_counts())

Brand
PEUGEOT          439237
VOLKSWAGEN       412498
SEAT             391586
RENAULT          364278
TOYOTA           342829
KIA              281884
HYUNDAI          271834
CITROEN          263510
OPEL             251996
FORD             240348
DACIA            239450
MERCEDES-BENZ    224494
AUDI             212268
NISSAN           191946
FIAT             183869
BMW              182245
SKODA            129012
MAZDA             83008
VOLVO             71663
JEEP              66798
MINI              44524
LEXUS             33936
MITSUBISHI        33339
LAND ROVER        26180
HONDA             25224
CUPRA             24927
SUZUKI            23424
DS                20876
ALFA ROMEO        16268
PORSCHE           11544
JAGUAR            10567
SSANGYONG          9987
SUBARU             9412
TESLA              9249
MG                 7644
Name: count, dtype: int64
Constructor
PSA-OPEL                         657067
VW GROUP PC                      584464
RENAULT-NISSAN-MITSUBISHI        487

In [12]:
# NaN status
# Calculate NaN counts and percentages for each column
nan_counts = df.isna().sum()
nan_percentages = (df.isna().sum() / len(df)) * 100

# Create a DataFrame to display the NaN information in a tidy format
nan_df = pd.DataFrame({
    'NaN Count': nan_counts,
    'NaN Percentage': nan_percentages
})

# Print the DataFrame
print(nan_df)

                        NaN Count  NaN Percentage
Country                         0             0.0
Constructor                     0             0.0
Veh_type                        0             0.0
Version                         0             0.0
Brand                           0             0.0
Veh_Model                       0             0.0
Veh_Category                    0             0.0
Kg_veh                          0             0.0
Test_mass                       0             0.0
CO2_wltp                        0             0.0
Wheelbase_mm                    0             0.0
Axle_width_steer_mm             0             0.0
Axle_width_other_mm             0             0.0
Energy                          0             0.0
Fuel_mode                       0             0.0
Engine_cm3                      0             0.0
Power_KW                        0             0.0
El_Consumpt_whkm                0             0.0
Erwltp (g/km)                   0             0.0


In [13]:
# adding new Emmissions on target column 1 - on target 0 over target
df['Em_on_target'] = (df['CO2_wltp'] <= 95).astype(int)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5151854 entries, 0 to 5409783
Data columns (total 24 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Country                 object 
 1   Constructor             object 
 2   Veh_type                object 
 3   Version                 object 
 4   Brand                   object 
 5   Veh_Model               object 
 6   Veh_Category            object 
 7   Kg_veh                  float64
 8   Test_mass               float64
 9   CO2_wltp                float64
 10  Wheelbase_mm            float64
 11  Axle_width_steer_mm     float64
 12  Axle_width_other_mm     float64
 13  Energy                  object 
 14  Fuel_mode               object 
 15  Engine_cm3              float64
 16  Power_KW                float64
 17  El_Consumpt_whkm        float64
 18  Erwltp (g/km)           float64
 19  year                    int64  
 20  Fuel consumption        float64
 21  Electric range (km)     float64
 22 

In [15]:
df.to_csv('ES_Cleaned.csv')